In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from logger import logger
import os
import cv2
import pydot
import tensorflow as tf
from tensorflow import keras
from ggplot import *
#-------------------------------------------------
#import tkeras.backend as K
#from tkeras.applications import inception_v3, inception_resnet_v2
#from tkeras import callbacks as Kcallbacks
#from tkeras.utils import plot_model
#from tkeras.models import Sequential, Model, load_model
#from tkeras.optimizers import Adam
#from tkeras.layers import Dense, Conv2D, Input, Activation, ZeroPadding2D, BatchNormalization,\
#    Flatten, MaxPooling2D, Dropout, concatenate, AveragePooling2D
#-------------------------------------------------------------------------
#from tensorflow import losses as tflosses
%matplotlib inline
np.random.seed(0)

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/usr/local/lib/python3.6/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/usr/local/lib64/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
train = np.load('./data/train/train_dataset.npy')
labels = np.load('./data/train/labels.npy')
print (train.shape, labels.shape, sep=' - ')

In [ ]:
def prepare_test_data() :
    test_dataset = np.load('./data/test/test_dataset.npy')
    test_ids = np.load('./data/test/ids.npy')
    print (test_dataset.shape, test_dataset, test_ids, sep=' - ')

In [ ]:
def get_more_images(imgs, epoches):
    
    more_images = []
    rotated_imgs = []
    for e in range(epoches) :
        for i in range(0,imgs.shape[0]):
            a=imgs[i,:,:,0]
            b=imgs[i,:,:,1]
            c=imgs[i,:,:,2]
            d=imgs[i,:,:,3]
            
            scale = np.random.random(1) * .6 + 1
            M = cv2.getRotationMatrix2D( (a.shape[0]/2,a.shape[1]/2), np.random.randint(0,360), scale)

            a1 = cv2.warpAffine(a, M, a.shape)
            b1 = cv2.warpAffine(b, M, a.shape)
            c1 = cv2.warpAffine(c, M, a.shape)
            d1 = cv2.warpAffine(d, M, a.shape)
            
            rotated_imgs.append(np.dstack((a1,b1,c1,d1)))
            
            
    r = np.array(rotated_imgs)
       
    more_images = np.concatenate((imgs,r))
    
    return more_images

e = 4
train = get_more_images(train, e)
labels = np.concatenate([labels for i in range(e+1)])
print (train.shape, labels.shape, sep=' - ')

In [ ]:
# getting validation set
train_dataset, valid_dataset, train_labels, valid_labels = train_test_split(train, labels,
                                                                            test_size=.0000001,shuffle=True)


print (train_dataset.shape, train_labels.shape, sep=' - ')
print (valid_dataset.shape, valid_labels.shape, sep=' - ')

del train, labels

------------------------

In [ ]:
plt.rcParams["figure.figsize"] = [15,15]
index = np.random.randint(0, train_dataset.shape[0])
f, axarr = plt.subplots(1,train_dataset.shape[3])
for i in range(train_dataset.shape[3]):
    _ = axarr[i].imshow(train_dataset[index,:,:,i])

print (" ** A ship" if train_labels[index,0] == 1 else " ** An iceberg")

In [ ]:
train_dataset = train_dataset[:,:,:,[0,3]]

In [ ]:
def convolution_layer(X, layer_name, paddings, convC, convF, convS, poolF, poolS, droprate=.2):
    X = keras.layers.ZeroPadding2D(paddings, name='Zero_Padding_'+layer_name)(X)
    X = keras.layers.Conv2D(convC, convF, strides=convS, name='Conv2d_'+layer_name)(X)
    X = keras.layers.BatchNormalization(axis=3, name='BN_'+layer_name)(X)
    X = keras.layers.Activation('relu', name='relu_activ_'+layer_name)(X)
    X = keras.layers.MaxPooling2D(poolF, poolS, name='MaxPooling_'+layer_name)(X)
    X = keras.layers.Dropout(droprate, name='dropout_'+layer_name)(X)
    return X
#--------------------------------------------------------
def fullyconnected_layer(X, layer_name, dense_units, activ, droprate=.2):
    X = keras.layers.Dense(dense_units, activation=activ, name='Dense_'+layer_name)(X)
    if layer_name == 'output' : return X
    X = keras.layers.BatchNormalization( name='BN_'+layer_name)(X)
    X = keras.layers.Dropout(droprate, name='dropout_'+layer_name)(X)
    return X
#-----------------------------------------------------------------------
def get_model(input_shape):
    X_input = Input(input_shape, name= 'input')
    # layer 1
    X = convolution_layer(X_input, 'layer1', 3, 64,  3, 1, 3, 2, .5)
    X = convolution_layer(X,       'layer2', 2, 128, 3, 1, 2, 2, .5)
    X = convolution_layer(X,       'layer3', 2, 128, 3, 1, 2, 2, .5)
    X = convolution_layer(X,       'layer4', 2, 64,  3, 1, 2, 2, .5)
    #-*-*-*-*-*-**-*-*-*-*-*-*-*
    X = Flatten()(X)
    X = fullyconnected_layer(X, 'FC1', 512, 'relu', .5)
    X = fullyconnected_layer(X, 'FC2', 256, 'relu', .5)
    X = fullyconnected_layer(X, 'FC3', 256, 'relu', .5)
    X = fullyconnected_layer(X, 'output', 1, 'sigmoid')
    
    model = Model(inputs=X_input, outputs=X)
    return model
model = get_model([75,75,2])
model.summary()

In [ ]:
model.compile(Adam(lr=.1), loss='binary_crossentropy',metrics=[])

In [ ]:
callbacks = [ 
    Kcallbacks.EarlyStopping(patience=10, verbose=1, mode='min'),
    Kcallbacks.History(),
    Kcallbacks.CSVLogger('./tmp/keras/checkpoints/2/logger1.log', append=True),
    Kcallbacks.ReduceLROnPlateau(factor=.1, patience=7, verbose=1),
    Kcallbacks.ModelCheckpoint('./tmp/keras/checkpoints/2/ckpt-{val_loss:.4f}-{epoch:02d}.hdf5',
                               verbose=1, save_best_only=1, mode='min')
] 

model.fit(x=train_dataset, y=train_labels, batch_size=32, epochs=100, shuffle=True, verbose=1, 
          validation_split=.2, callbacks=callbacks)

In [ ]:
model.evaluate(train_dataset[2500:2700], train_labels[2500:2700])

In [ ]:
best_model.evaluate(train_dataset[2500:2700], train_labels[2500:2700])

In [ ]:
def predict_test_dataset(model, subid):
    test_data, test_ids = prepare_test_data()
    test_preds = model.predict(test_data, batch_size=200, verbose=1)
    sub_df = pd.DataFrame({'id':test_ids, 'is_iceberg':test_preds.reshape((-1))})
    sub_df.to_csv('./tmp/keras/checkpoints/1/submission-{}'.format(subid), index=False)

In [ ]:
predict_test_dataset(model, '.0721')

In [ ]:
best_model = load_model('./tmp/keras/checkpoints/1/ckpt-0.2062-34.hdf5',
                        custom_objects={'log_loss':tflosses.log_loss})

In [ ]:
plot_model(best_model, to_file='./model.png')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

------------------------------------

In [ ]:
X_input = Input([75,75,4], name='X_input')
model = inception_v3.InceptionV3(input_tensor=X_input, pooling='max', classes=1, weights=None)

In [ ]:
inc_model.summary()

In [ ]:
np.savetxt('./data/train/train_ready.csv', train_dataset.reshape(-1,75*75*2), delimiter=',')
np.savetxt('./data/train/labels_ready.csv', train_labels, delimiter=',')

In [ ]:
train_dataset.shape

In [ ]:
training_data = tf.contrib.learn.datasets.base.load_csv_with_header('./data/train/labels_ready.csv',
                                                                    target_dtype=np.float, features_dtype=np.float)

In [ ]:
X = tf.data.TextLineDataset('./data/train/train_ready.csv')

In [ ]:
ite = X.make_one_shot_iterator().get_next()